In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import load_data, load_config, write_to_file, one_hot_encoding

# Load configuration
config = load_config('./config.yaml')

# Load the data and reshape from (32 x 32) to (1024 x 1)
x_train, y_train, x_test, y_test = load_data()

# One-hot encoding
y_train = np.eye(len(y_train), 10)[y_train]
y_test = np.eye(len(y_test), 10)[y_test]

x_train = np.array([image.reshape((1024)) for image in x_train], dtype='float')
x_test = np.array([image.reshape((1024)) for image in x_test], dtype='float')

# Create validation set out of training data.
num = int(len(x_train) * 0.8)
[x_train, x_val]= np.split(x_train, [num])
[y_train, y_val] = np.split(y_train, [num])

In [2]:
# Calculate feature mean and standard deviation for x_train, and use them to
# Z score x_train, X_val and X_test
def z_score_train_test(train, val, test):
    train_T = train.T
    val_T = val.T
    test_T = test.T
    for i in range(len(train_T)):
        mean = np.mean(train_T[i])
        SD = np.std(train_T[i])
        train_T[i] = (train_T[i] - mean) / SD
        val_T[i] = (val_T[i] - mean) / SD
        test_T[i] = (test_T[i] - mean) / SD
    return train_T.T, val_T.T, test_T.T

# Z-scoring
x_train, x_val, x_test = z_score_train_test(x_train, x_val, x_test)

In [3]:
train_acc = []
valid_acc = []
train_loss = []
valid_loss = []
best_model = None

In [4]:
from neuralnet import *

model = NeuralNetwork(config=config)

In [17]:
def accuracy(y, t):
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)
    res = [y_hat == t_hat for y_hat, t_hat in zip(y, t)]
    return np.sum(res) / len(res)

In [18]:
# SGD
for i in range(config['epochs']): 
    # Randomize the order of the indices into the training set
    shuffled_indices = np.random.permutation(len(x_train))
    x_train = x_train[shuffled_indices]
    y_train = y_train[shuffled_indices]
    for j in range(0, len(x_train), config['batch_size']):
        if (j + config['batch_size'] < len(x_train)):
            batch_x = x_train[[j, j + config['batch_size']]]
            batch_y = y_train[[j, j + config['batch_size']]]
        else:
            batch_x = x_train[[j, len(x_train) - 1]]
            batch_y = y_train[[j, len(x_train) - 1]]
        y, loss = model(x=batch_x, targets=batch_y)
        model.backward()
        
    y, loss = model.forward(x_val, y_val)
    acc = accuracy(y, y_val)
    print('Epoch', i, 'Loss', loss, 'Accuracy', acc)

Epoch 0 Loss 2.5133147001142744 Accuracy 0.1291973791973792
Epoch 1 Loss 2.452964720109597 Accuracy 0.14182364182364182
Epoch 2 Loss 2.4196881822051455 Accuracy 0.14905814905814907
Epoch 3 Loss 2.400072424734549 Accuracy 0.15226590226590225
Epoch 4 Loss 2.38648416030877 Accuracy 0.15779415779415779
Epoch 5 Loss 2.371269238193037 Accuracy 0.15124215124215123
Epoch 6 Loss 2.3585589122797375 Accuracy 0.1558149058149058
Epoch 7 Loss 2.3510655346001186 Accuracy 0.1650969150969151
Epoch 8 Loss 2.345583625894367 Accuracy 0.1622986622986623
Epoch 9 Loss 2.340652850851343 Accuracy 0.16223041223041224
Epoch 10 Loss 2.33269951182659 Accuracy 0.16755391755391755
Epoch 11 Loss 2.327833777502246 Accuracy 0.16325416325416325
Epoch 12 Loss 2.3252229227625203 Accuracy 0.16257166257166258
Epoch 13 Loss 2.3221006480011823 Accuracy 0.16516516516516516
Epoch 14 Loss 2.316696313790157 Accuracy 0.17553917553917553
Epoch 15 Loss 2.3119441608075504 Accuracy 0.1788151788151788
Epoch 16 Loss 2.307439316017384 Ac

In [27]:
model = NeuralNetwork(config=config)

# Choose one output bias weight'
epsilon = 1e-2
model.layers[2].b[8] += epsilon
model.forward(x_val)
e_w_plus

1